This script aims to 
- load `T_air` raster outputs and perform zonal stats (`mean`) by bourough

# Load data

## Function 

### function - short name

In [33]:
import re
from pathlib import Path

def make_short_name(path: Path, base_dir: Path) -> str:
    """
    Generate a compact name from a long UCM scenario file.

    Example:
    input:
        dir_ucm_out / "current_lulc" / "work_and_energy_runs" / "intermediate"
        / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif"
    output:
        "current_lulc_20_2"
    """
    path = Path(path)

    # 1. Get the scenario prefix (folder immediately under base_dir)
    try:
        prefix = path.relative_to(base_dir).parts[0]
    except ValueError:
        raise ValueError(f"{path} is not inside {base_dir}")

    # 2. Extract the deg/uhi numbers from the filename
    # m = re.search(r"scenario(?:\d+)?_([\d.]+)deg_([\d.]+)uhi", path.name)
    m = re.search(r"scenario(?:\d+)?(?:_[A-Za-z0-9]+)*_([\d.]+)deg_([\d.]+)uhi", path.name)
    if not m:
        raise ValueError(f"Could not parse scenario from {path.name}")

    deg = str(int(float(m.group(1))))   # "20.0" -> "20"
    uhi = str(int(float(m.group(2))))   # "2.0"  -> "2"

    return f"{prefix}_{deg}_{uhi}"


# # Example usage
# from pathlib import Path

# wd_main     = Path(r"G:\Shared drives\Wellcome Trust Project Data") 
# dir_ucm_out = wd_main / "2_postprocess_intermediate" / "UCM_official_runs"

# f = dir_ucm_out / "current_lulc" / "work_and_energy_runs" / "intermediate" \
#     / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif"

# print(make_short_name(f, dir_ucm_out))
# # -> current_lulc_20_2


### function - zonal stats

In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from rasterstats import zonal_stats

import math                              # EDIT
import numpy as np                       # EDIT
import rasterio as rio

def run_zonal_stats(
    aoi_path,
    raster_path,
    model_output_path, 
    out_dir,
    stats=["mean"],
    nodata=-9999.0,
    suffix=None,
    drop_cols=None
):
    """
    Compute zonal statistics for a single raster and save results to CSV.

    Parameters
    ----------
    aoi_path : str or Path
        Path to AOI shapefile/GeoJSON/GeoPackage.
    raster_path : str or Path
        Path to raster file.
    out_dir : str or Path
        Directory where output CSV will be saved.
    stats : list[str], default ["mean"]
        Zonal statistics to compute.
    nodata : float, default -9999.0
        NoData value in raster.
    suffix : str or None
        Optional suffix to append to stat column names and output file.
        If None, uses raster file stem.
    drop_cols : list[str] or None
        Columns to drop from output DataFrame.

    Returns
    -------
    pd.DataFrame
        DataFrame with zonal statistics.
    """
    aoi = gpd.read_file(aoi_path)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    raster_path = Path(raster_path)
    suffix = make_short_name(raster_path, model_output_path)

    
    # look for folder names of interest
    candidates = ["lulc", "scenario"]
    lc_scenario = next((p for p in raster_path.parts if any(c in p for c in candidates)), None)
    print(lc_scenario)  # → "scenario1"




    # -------- Detect NoData from raster metadata (robust) --------  # EDIT
    with rio.open(raster_path) as ds:                                 # EDIT
        meta_nodata = ds.nodata if ds.nodata is not None else ds.nodatavals[0]  # EDIT
        dtype = ds.dtypes[0].lower()                                   # EDIT

    # Prefer the raster's own NoData; otherwise smart fallback          # EDIT
    nodata_to_use = meta_nodata if meta_nodata is not None else nodata # EDIT

    # If dtype is float32 and nodata is still None, many rasters imply the sentinel -FLT_MAX  # EDIT
    if (meta_nodata is None) and ("float32" in dtype):                 # EDIT
        nodata_to_use = np.finfo(np.float32).min                       # EDIT

    # If nodata_to_use somehow ended up NaN, rasterstats may not like it as a parameter.       # EDIT
    # In that case, skip passing nodata explicitly; rasterio will still mask NaNs on read.     # EDIT
    pass_nodata = not (isinstance(nodata_to_use, float) and math.isnan(nodata_to_use))  # EDIT




    # -------------------- Run zonal stats --------------------
    zs_kwargs = dict(geojson_out=True, stats=stats)                    # EDIT
    if pass_nodata:                                                    # EDIT
        zs_kwargs["nodata"] = nodata_to_use                            # EDIT

    zs_result = zonal_stats(
        aoi,
        str(raster_path),
        **zs_kwargs
    )

    zs_gdf = gpd.GeoDataFrame.from_features(zs_result)
    df = pd.DataFrame(zs_gdf)

    # Add scenario/lc column if found
    if lc_scenario:
        df["lc_scenario"] = lc_scenario

    # add raster name column
    df["raster"] = raster_path.name

    # # Rename stats columns
    # rename_dict = {s: f"{s}_{suffix}" for s in stats}
    # df = df.rename(columns=rename_dict)

    

    # Drop unwanted columns
    if drop_cols:
        df = df.drop(columns=drop_cols, errors="ignore")

    # Save to CSV
    out_csv = out_dir / f"{suffix}_zonal_stats.csv"
    df.to_csv(out_csv, index=False)
    print(f"\t[OK] Zonal stats saved → {out_csv}")
    print(f"\t[info] dtype={dtype}, nodata_used={nodata_to_use!r}")      # EDIT

    return df


## Data dir

In [35]:
from pathlib import Path

# Set the working directory
wd_main     = Path(r"G:\Shared drives\Wellcome Trust Project Data") 
wd_shp      = wd_main / "1_preprocess" / "UrbanCoolingModel" / "OfficialWorkingInputs" / "AOIs"
dir_ucm_out = wd_main / "2_postprocess_intermediate" / "UCM_official_runs"
dir_ucm_stat= dir_ucm_out / "ucm_output_postprocess"
figures_dir = wd_main / "3_final" / "UCM_figures"

## check nodata value

In [36]:
# Example raster path
import rasterio as rio
from rasterio.enums import MaskFlags

path = dir_ucm_out / "scenario41"   / "work_and_energy_runs"  / 'tcc_10prc' / "intermediate" / "T_air_london_scenario4_10prc_22deg_2uhi_55hum_energy_productivity.tif"
# path = dir_ucm_out / "scenario0" / "work_and_energy_runs" / "intermediate" / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif"

with rio.open(path) as ds:
    # 1) Reported NoData (per band)
    print("nodata (band 1):", ds.nodatavals[0])  # or ds.nodata (alias for band 1)

    # 2) Does the band use an internal mask instead of a nodata value?
    print("mask flags:", ds.mask_flag_enums[0])  # e.g., [MaskFlags.per_dataset, MaskFlags.alpha]

    # 3) Is there an alpha band?
    print("count (bands):", ds.count)

    # 4) Quick sanity on what’s actually masked
    m = ds.read_masks(1)                 # 0 = masked, 255 = valid
    print("masked pixels:", (m == 0).sum())


nodata (band 1): -3.4028234663852886e+38
mask flags: [<MaskFlags.nodata: 8>]
count (bands): 1
masked pixels: 10385702


## Batch temp raster 

In [37]:
aoi_adm = wd_shp / "London_Borough_aoi.shp"  # Administrative boundary (e.g., census tracts)

drop_cols = ["HECTARES", "NONLD_AREA", "ONS_INNER", "SUB_2009", "SUB_2006", "geometry"]


## loop more rasters

In [38]:

rasters = [
    ## s0
    dir_ucm_out / "scenario0" / "work_and_energy_runs" / "intermediate" / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif",
    dir_ucm_out / "scenario0" / "work_and_energy_runs" / "intermediate" / "T_air_london_scenario_22.0deg_2.0uhi_55.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario0" / "work_and_energy_runs" / "intermediate" / "T_air_london_scenario_25.0deg_5.0uhi_45.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario0" / "work_and_energy_runs" / "intermediate" / "T_air_london_scenario_28deg_5uhi_45hum_energy_productivity.tif",

    ## s1
    dir_ucm_out / "scenario1"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif",
    dir_ucm_out / "scenario1"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_22.0deg_2.0uhi_55.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario1"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_25.0deg_5.0uhi_45.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario1"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_28deg_5uhi_45hum_energy_productivity.tif",
    
    ## s2
    dir_ucm_out / "scenario2"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif",
    dir_ucm_out / "scenario2"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_22.0deg_2.0uhi_55.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario2"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_25.0deg_5.0uhi_45.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario2"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_28deg_5uhi_45hum_energy_productivity.tif",

    ## s3
    dir_ucm_out / "scenario3"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_20.0deg_2.0uhi_66.9hum_energy_productivity.tif",
    dir_ucm_out / "scenario3"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_22.0deg_2.0uhi_55.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario3"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario_25.0deg_5.0uhi_45.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario3"   / "work_and_energy_runs"  / "intermediate" / "T_air_london_scenario3_28deg_5uhi_45hum_energy_productivity.tif",

    ## s4.10
    dir_ucm_out / "scenario41"   / "work_and_energy_runs"  / 'tcc_10prc' / "intermediate" / "T_air_london_scenario4_10prc_22deg_2uhi_55hum_energy_productivity.tif",
    dir_ucm_out / "scenario41"   / "work_and_energy_runs"  / 'tcc_10prc' / "intermediate" / "T_air_london_scenario4_10prc_25deg_5uhi_45hum_energy_productivity.tif",
    dir_ucm_out / "scenario41"   / "work_and_energy_runs"  / 'tcc_10prc' / "intermediate" / "T_air_london_scenario4_10prc_28deg_5uhi_45hum_energy_productivity.tif",

    ## s4.20
    dir_ucm_out / "scenario42"   / "work_and_energy_runs"  / 'tcc_20prc' / "intermediate" / "T_air_london_scenario4_20prc_22.0deg_2.0uhi_55.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario42"   / "work_and_energy_runs"  / 'tcc_20prc' / "intermediate" / "T_air_london_scenario4_20prc_25.0deg_5.0uhi_45.0hum_energy_productivity.tif",
    dir_ucm_out / "scenario42"   / "work_and_energy_runs"  / 'tcc_20prc' / "intermediate" / "T_air_london_scenario4_20prc_28deg_5uhi_45hum_energy_productivity.tif",
]


all_results = []
for raster in rasters:
    # out_dir = raster.parent
    df = run_zonal_stats(
        aoi_path=aoi_adm,
        raster_path=raster,
        model_output_path = dir_ucm_out,
        out_dir=dir_ucm_stat,
        stats=["mean"],
        drop_cols=drop_cols
    )
    all_results.append(df)

# # Optionally merge all results side-by-side
# df_combined = pd.concat(all_results, axis=1)
# print(df_combined.head())


# row-wise combine
df_combined = pd.concat(all_results, axis=0, ignore_index=True)


scenario0
[OK] Zonal stats saved → G:\Shared drives\Wellcome Trust Project Data\2_postprocess_intermediate\UCM_official_runs\ucm_output_postprocess\scenario0_20_2_zonal_stats.csv
[info] dtype=float64, nodata_used=-3.4028234663852886e+38
scenario0
[OK] Zonal stats saved → G:\Shared drives\Wellcome Trust Project Data\2_postprocess_intermediate\UCM_official_runs\ucm_output_postprocess\scenario0_22_2_zonal_stats.csv
[info] dtype=float64, nodata_used=-3.4028234663852886e+38
scenario0
[OK] Zonal stats saved → G:\Shared drives\Wellcome Trust Project Data\2_postprocess_intermediate\UCM_official_runs\ucm_output_postprocess\scenario0_25_5_zonal_stats.csv
[info] dtype=float64, nodata_used=-3.4028234663852886e+38
scenario0
[OK] Zonal stats saved → G:\Shared drives\Wellcome Trust Project Data\2_postprocess_intermediate\UCM_official_runs\ucm_output_postprocess\scenario0_28_5_zonal_stats.csv
[info] dtype=float64, nodata_used=-3.4028234663852886e+38
scenario1
[OK] Zonal stats saved → G:\Shared drives\

## save all summarized data

In [ ]:
## 1. directly save if run all the rasters at once
out_csv = dir_ucm_stat / f"T_air_london_{len(all_results)}_zonal_stats_long.csv"
df_combined.to_csv(out_csv, index=False)

print(f"[OK] Zonal stats saved → {out_csv}")

[OK] Zonal stats saved → G:\Shared drives\Wellcome Trust Project Data\2_postprocess_intermediate\UCM_official_runs\ucm_output_postprocess\T_air_london_22_zonal_stats_long.csv


In [ ]:

## 2. save after running multiple rasters <- need to manually bind all results

# ==== CONFIG ====
ROOT = dir_ucm_stat   # ← change me
PATTERN = "scenario*_zonal_stats.csv" # start with "scenario", end with "_zonal_stats.csv"
RECURSIVE = False                      # set False to only scan top-level
# ================

def parse_scenario(filename: str) -> str:
    """
    Extract the 'scenario...' block from filenames like:
      scenario4_10prc_zonal_stats.csv
      scenario_foo_bar_zonal_stats.csv
      scenario3_zonal_stats.csv
    """
    m = re.search(r"(scenario(?:\d+)?(?:_[A-Za-z0-9]+)*)_zonal_stats\.csv$", filename)
    return m.group(1) if m else "scenario"

# 1) Find matching files
files = sorted((ROOT.rglob if RECURSIVE else ROOT.glob)(PATTERN))

if not files:
    raise FileNotFoundError(f"No files matching {PATTERN} under {ROOT}")

# 2) Read & bind
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append(
        df.assign(
            source=str(f),
            scenario=parse_scenario(f.name)
        )
    )

combined = pd.concat(dfs, ignore_index=True, sort=False)

## 3) Save combined as CSV
out_csv = ROOT / f"T_air_london_{len(files)}_zonal_stats_long.csv"
combined.to_csv(out_csv, index=False)

# Optional: quick sanity checks
print(f"Loaded {len(files)} files → {len(combined):,} rows")

# combined is your final DataFrame
combined.head()


print(f"[OK] Zonal stats saved → {out_csv}")

# Plot

This takes a while to run ... 

In [40]:
# import re
# import numpy as np
# import rasterio
# import matplotlib.pyplot as plt
# from pathlib import Path

# # ---------- helper to build compact label e.g. "current_lulc_20_2" ----------
# def make_short_name(path: Path, base_dir: Path) -> str:
#     path = Path(path)
#     try:
#         prefix = path.relative_to(base_dir).parts[0]  # e.g., current_lulc / scenario2 / scenario3
#     except ValueError:
#         prefix = path.parent.name  # fallback to immediate parent if path not under base_dir
#     m = re.search(r"scenario_([\d.]+)deg_([\d.]+)uhi", path.name)
#     if not m:
#         return prefix
#     deg = str(int(float(m.group(1))))
#     uhi = str(int(float(m.group(2))))
#     return f"{prefix}_{deg}_{uhi}"

# # ---------- config ----------
# # Put your 'dir_ucm_out' here if not already defined:
# # from pathlib import Path
# # wd_main     = Path(r"G:\Shared drives\Wellcome Trust Project Data") 
# # dir_ucm_out = wd_main / "2_postprocess_intermediate" / "UCM_official_runs"


# BASE_DIR = dir_ucm_out
# MAX_SAMPLES_PER_RASTER = 200_000   # cap for speed/memory; raise/lower as you wish

# # ---------- load values ----------
# violin_data = []
# labels = []

# for rp in rasters:
#     rp = Path(rp)
#     if not rp.exists():
#         print(f"[warn] missing raster: {rp}")
#         continue

#     with rasterio.open(rp) as src:
#         arr = src.read(1)  # first band
#         nodata = src.nodata

#     # mask nodata / nan / inf
#     arr = arr.astype("float64", copy=False)
#     if nodata is not None:
#         valid = (arr != nodata)
#     else:
#         valid = np.ones(arr.shape, dtype=bool)

#     valid &= np.isfinite(arr)  # drop NaN, +inf, -inf
#     vals = arr[valid].ravel()

#     # optional subsample for performance
#     if vals.size > MAX_SAMPLES_PER_RASTER:
#         idx = np.random.default_rng(42).choice(vals.size, size=MAX_SAMPLES_PER_RASTER, replace=False)
#         vals = vals[idx]

#     if vals.size == 0:
#         print(f"[warn] all values invalid after masking: {rp.name}")
#         continue

#     violin_data.append(vals)
#     labels.append(make_short_name(rp, BASE_DIR))

# # ---------- plot (matplotlib only; one figure; no custom colors) ----------
# fig, ax = plt.subplots(figsize=(10, 6))

# parts = ax.violinplot(
#     violin_data,
#     showmeans=True,
#     showextrema=True,
#     showmedians=True
# )

# # Style the parts
# # Means are red dashed lines
# if 'cmeans' in parts:
#     parts['cmeans'].set_color('red')
#     parts['cmeans'].set_linestyle('--')
#     parts['cmeans'].set_linewidth(2)

# # # Medians are blue solid lines
# # if 'cmedians' in parts:
# #     parts['cmedians'].set_color('blue')
# #     parts['cmedians'].set_linewidth(2)

# # X-axis labels
# ax.set_xticks(np.arange(1, len(labels) + 1))
# ax.set_xticklabels(labels, rotation=30, ha="right")

# ax.set_xlabel("Scenario")
# ax.set_ylabel("Raster values")
# ax.set_title("Scenario comparison (violin plot)")

# plt.tight_layout()
# plt.show()
